In [ ]:
import numpy as np
import pickle
import os
from PIL import ImageDraw,Image,ImageFont

In [ ]:
def nms(dets, thresh):
    # 所有box的坐标信息。注意这里是数组而不是列表
    # x1 y1 左上角角坐标；  x2 y2 右下角坐标
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    # 计算出所有box的面积;图片评分（置信度）按降序排序
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]  # 注意这里orders是边界框的索引

    # 保留最后需要保留的边框的索引
    keep = []
    while order.size > 0:
        # order[0]是目前置信度最大的，肯定保留; i是还未处理的图片中的最大评分索引
        i = order[0]
        # 保留改图片的值
        keep.append(i)

        # 计算窗口i与其他窗口的交叠的面积
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        # 计算相交框的面积,不相交时用0代替
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h

        # 计算IOU：相交的面积/相并的面积
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        # 只保留比例小于阙值的box，然后继续处理,因为这可能是另外一个目标
        inds = np.where(ovr < thresh)[0]
        order = order[inds + 1]

    return keep

In [4]:
# YOLO-V5
file_from_yolom = './yoloyoloface/runs/detect/m/labels'
# RCNN_MMDetection_pkl
pkl_from_RCNN = './RCNN/FasterRCNN/fasterRCNN_result.pkl'
data_pkl = pickle.load(open(pkl_from_RCNN,"rb"))

# image file
img_file = f'./Datasets/widerface_YOLO/images/val'
image_list = os.listdir(img_file)

#阈值 越高越不会合并
thresh = 0.75

# 记录一些YOLOV5没有检测出框的文件名
notfound_list = []
# 最后的结果
result_keeped = []

for idx, i in enumerate(image_list):
    # record result
    result_t = []
    
    img_name = image_list[idx].split('.')[0]

    # img_yolo_info
    img_jpg = f'./Datasets/widerface_YOLO/images/val/{img_name}.jpg'
    box_from_yolom = f'./yoloyoloface/runs/detect/m/labels/{img_name}.txt'
    img = Image.open(img_jpg)
    width, height = img.width, img.height

    try:
        with open(box_from_yolom, mode='r',encoding='utf-8') as f:
            line = f.read().splitlines()
            for i in line:
                _, center_x, center_y, w, h = map(float, i.split(' '))

                x_min, y_min, x_max, y_max = center_x - 0.5*w, center_y - 0.5*h , center_x + 0.5*w, center_y + 0.5*h
                x_min, y_min, x_max, y_max = x_min*width, y_min*height, x_max*width, y_max*height 
                result_t.append([x_min, y_min, x_max, y_max, 1.0])
    except FileNotFoundError:
        # do nothing
        notfound_list.append(box_from_yolom)

    # RCNN_info
    for i in data_pkl[idx][0]:
        result_t.append(list(i))
        
    # NMS 进行mmdetection结果与YOLOV5结果的合并NMS
    result_t = np.array(result_t).reshape(-1,5)
    keep_dets = nms(result_t, thresh)
    result_keeped.append([result_t[keep_dets]])

In [6]:
save_file = open('./after_NMS_{}.pkl'.format(thresh), 'wb')
pickle.dump(result_keeped, save_file)